# Developing Custom GRASS Tools - FOSS4G 2022 Workshop

Learn how to develop custom tools (aka addons or modules) for GRASS GIS in Python and, if you like, in C.

Python scripting is powerful, but what is even more powerful is turning a script into a GRASS tool with just a few tricks and tweaks we will cover in this workshop. When you develop a GRASS tool (aka module), you get a graphical user interface (GUI), command line interface, and convenience you and your users will appreciate. Such tools can be published in a community-maintained addon repository which helps not only to distribute the tool, but also to maintain the code in the long term.

We will focus on Python, but we will cover tools written in C, too, because even compiled tools in C and C++ can be in this community-maintained repository and distributed to users.

## Authors

### Vaclav (Vashek) Petras

Vashek is a research software engineer at Center for Geospatial Analytics, NC State University. He has been a member of the GRASS GIS development team since 2012 and currently serves on its project steering committee. He has a Master's degree in Geoinformatics from Czech Technical University and a PhD in Geospatial Analytics from NC State University.

### Corey White

Corey is the founder of OpenPlains Inc. and a researcher at Center for Geospatial Analytics, NC State University. He has been a GRASS GIS contributor since 2020. He has Master of Geographic Information Systems and Technology and PhD in Geospatial Analytics both from NC State University.

## Related talks

* _Parallelization Tips for Geoprocessing with GRASS GIS_, Tuesday, 10:30 am – 11:00 am
* _An Automated Method for LiDAR-derived Impoundment Water Level Elevation_, Tuesday, 10:30 AM – 11:00 AM
* _GRASS GIS: Not What You Think_, Wednesday, 10:30 am – 11:00 am
* _Computational Notebooks for Reproducible Geospatial Computation in Research and Education_, Wednesday, 2:00 pm – 2:30 pm

## Outline

- This notebook
  * Python script structure
  * Running Python scripts
  * Running GRASS GIS
- Tools for GRASS GIS in Python
- Best practices for writing GRASS tools
- Best practices for writing GRASS tools
- Tools for GRASS GIS in C

## Workshop Software Setup

The workshop material assumes it runs in the prepared Binder environment which is running Ubuntu and GRASS GIS is already installed there.

The Binder is set up with development version of GRASS GIS 8.3, but the notebooks will work with 8.2 as well.

## Getting GRASS GIS Ready

If you are running the notebook in the prepared Binder, there is nothing to do. If you are compiling GRASS GIS yourself, start JupyterLab in a way that your compiled GRASS GIS is the first _grass_ command on the path, e.g.,:

```bash
PATH=~/grass/code/bin.x86_64-pc-linux-gnu/:$PATH jupyter lab
```

For other cases, please refer to [GRASS GIS Jupyter notebooks wiki page](https://grasswiki.osgeo.org/wiki/GRASS_GIS_Jupyter_notebooks#Running_a_Jupyter_notebook_locally).

Check that GRASS GIS is running and that you get the expected version:

In [ ]:
!grass --version

## Getting Data Ready

The Binder setup already has the [North Carolina full sample dataset](https://grass.osgeo.org/sampledata/north_carolina/nc_spm_grass7.zip) included.

To do our test runs in an isolated environment, we will create a new mapset (aka subproject) called _foss4g_:

In [ ]:
!grass -e -c ~/grassdata/nc_spm_08_grass7/foss4g

To start over later on, you can use a different mapset or delete this one using:

```bash
rm -r ~/grassdata/nc_basic_spm_grass7/foss4g
```

If you need to download the data, you can use the following command in Bash:

```bash
grass --tmp-location XY --exec g.download.location url=https://grass.osgeo.org/sampledata/north_carolina/nc_spm_grass7.zip path=~/grassdata/
```

## Python Script

Let's start with a basic Python script which uses GRASS GIS. GRASS Python packages are usually not on Python path, so we will use GRASS command line interface to get the path to these packages before we import them.

The script starts a GRASS session and uses the mapset we created above. Then, it prints the current mapset name. 

## Full Python Script Structure

The best practice for Python scripts is to use a _main_ function which is called from the so-called "if name equals main" block. The name of the of the _main_ function is not import, although it usually is _main_, while the syntax of the if-name-equals-main block is. Generally, all code should be in the _main_ function or called from it. This is the structure we will use from now on:

```python
def main():
    pass

if __name__ == "__main__":
    main()
```

In [ ]:
%%writefile hello_world.py
#!/usr/bin/env python
# The line above (shebang) tells unix OSs how to execute script.

# Imports make installed packages accessible in the script.
import sys


# All functionality is called from the function called main.
def main():
    # "Actual" code is here in this function or ther functions.
    print(sys.argv)


if __name__ == "__main__":
    # This tells Python what should be done when running the script.
    main()

In [ ]:
!python hello_world.py

In [ ]:
!python hello_world.py "Hello world!"

In [ ]:
!chmod u+x hello_world.py

In [ ]:
%%writefile print_mapset_script.py
#!/usr/bin/env python
import subprocess
import sys

# bootstrap for importing GRASS packages 
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# GRASS-specific imports
import grass.script as gs
import grass.script.setup  # This line is needed only for v8.2 and older.


def main():
    # Starts a GRASS session and uses its context manager API.
    with grass.script.setup.init("~/grassdata/nc_basic_spm_grass7/foss4g") as session:
        print(gs.read_command("g.mapset", flags="p"))


if __name__ == "__main__":
    main()

In [ ]:
!chmod u+x print_mapset_script.py

In [ ]:
!./print_mapset_script.py "Hello world!"

## Running Python Scripts from Command Line

For testing a script and for integrating it in GRASS GIS, it is advantageous to see how a script is executed from command line or generally as a subprocess.

Before, we used IPython kernel cell magic `%%python` to run a cell as a separate Python script. Now, we will use `%%writefile` cell magic to create a Python file which we will execute in the following cell.

Use _python_ to execute the script. It's name (or path) are provided as parameter:

## Executable Scripts and Shebang

On unix-like systems (Linux, macOS, ...), specifying the Python interpreter can be avoided when the script has execute permissions and the first line of the script called shebang specifies which interpreter to use for the given script. A minimal script then looks like this:

The first line now caries very special meaning, but for Python it is just a comment, although some helper tools may recognize it.

Let's add shebang to our script:

Permissions are managed using _chmod_. `chmod u+x` makes a file executable for the user:

Script can then run without specifying the Python interpreter:

Note `./` which says the script is in the current directory. The path is always mandatory in this case even if it is the current directory. Installed GRASS tools are _on path_, i.e., are where the operating system looks for executables, so for installed tools, no path needs to be specified.

The executable mechanism on Windows is different and GRASS GIS does number of steps to ensure that the scripts can be executed and right Python is used.

## Running in GRASS GIS

GRASS tools are different from Python scripts which are using GRASS GIS in the way that they are not setting up their own GRASS session. The tools are already running in a session which was previously set up by the user in some interactive or automated way, e.g., using GUI in a desktop environment or from a Python script.

The following script assumes that it runs in a GRASS session. Because we separated the concern about the GRASS session, the script is simpler: There is no need to set up path to GRASS packages and initialize GRASS session with a mapset.

In [ ]:
%%writefile print_mapset.py
#!/usr/bin/env python

import sys

import grass.script as gs


def main():
    print(gs.read_command("g.mapset", flags="p"))


if __name__ == "__main__":
    main()

The script can then run in an interactive GRASS session (from GUI or shell) or it can be executed using the `--exec` interface:

In [ ]:
!grass ~/grassdata/nc_basic_spm_grass7/foss4g --exec python ./print_mapset.py

Let's make the script executable:

In [ ]:
!chmod u+x print_mapset.py

For the executable script, we can leave out `python`:

In [ ]:
!grass ~/grassdata/nc_basic_spm_grass7/foss4g --exec ./print_mapset.py

## Tool for GRASS GIS

<div class="alert alert-info">
Scripts like all other programs, can take command line parameters. This is a crucial feature for developing general scripts and GRASS tools.
</div>

Since GRASS tools are executable scripts (or generally programs), the interface of a GRASS tool is the command line interface of a Python script. A dedicated function _grass.script.parser_ takes care of processing the command line arguments based on the interface description specified in a Python comment with a key-value syntax defined by GRASS GIS.

Here is a simple script which prints parameters received on the command line:

In [ ]:
%%writefile print_mapset.py
#!/usr/bin/env python

# %module
# % description: Prints its raw parameters
# %end

import sys

import grass.script as gs

def main():
    gs.parser()
    print(f"Parameters are: {sys.argv}")


if __name__ == "__main__":
    main()

Make the script executable:

In [ ]:
!chmod u+x print_mapset.py

Try different combinations of parameters:

The script works just the same with `grass ... --exec` where parameters go after the script:

In [ ]:
!grass ~/grassdata/nc_basic_spm_grass7/foss4g --exec ./print_mapset.py --help